<a href="https://colab.research.google.com/github/mudogruer/airtuerk_chatbot_model_dev/blob/main/AirTuerk_ChatBot_AI_models_design.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  **Introduction: Airtuerk AI Chatbot Açıklaması**

Bu Jupyter Notebook, **Airtuerk AI Chatbot** projesini içeriyor. Chatbot, **OpenAI'nin GPT modellerini** ve **LangChain** frameworkü kullanarak LLM tabanlı bir uçuş asistanı olarak çalışır. Kullanıcıdan aldığı girdilere göre uçuş bilgilerini çıkarır, filtreler ve uygun uçuşları listeleyerek interaktif bir deneyim sunar.

## **Ana Bileşenler**
### 1. **Dependencies Kurulumu**
   - OpenAI, LangChain, FAISS gibi kütüphaneler yüklenir.

### 2. **Gerekli kütüphanelerin yüklenmesi**
  - Gelistirme süreci boyunca kullanilacak tüm kütüphaneleri burada yüklüyoruz.

### 3. **Data Yükleme ve İşleme**
   - **Uçuş verileri** bir Excel dosyasından alınır.
   - **Metin verileri** işlenerek bir bilgi tabanı oluşturulur.

### 4. **LLM**
   - Kullanıcının uçuşla ilgili mesajlarını analiz eder.
   - Uçuş detaylarını çıkarır (kalkış, varış, tarih vb.).
   - FAISS ile belgeleri vektörleştirerek daha verimli arama yapar.

### 5. **Tarih İşleme ve Uçuşları Filtreleme**
   - Kullanıcının belirttiği tarih ve lokasyonlara göre uygun uçuşları listeler.

### 6. **Gerçek Zamanlı Chatbot**
   - Kullanıcıdan giriş alarak **uçuş rezervasyonu** yapmak için yönlendirir.

Bu chatbot, uçuş bilgilerini anlama, uygun seçenekleri sunma ve bir diyalog akışı içinde kullanıcıyla etkileşim kurma yeteneğine sahiptir.

## **1. Dependencies Kurulumu**

**📌** **📌**İlk olarak, chatbotun çalışması için gerekli kütüphaneleri yüklüyoruz. Bu kütüphaneler, **Large Language Modeller, vektör tabanlı arama ve OpenAI API entegrasyonu** için gereklidir.

### **Kurulan Kütüphaneler:**
- `openai`: OpenAI'nin GPT modellerini kullanmak için.
- `langchain`: Büyük dil modelleriyle (LLM) zincirleme işlem yapmak için.
- `langchain_community`: LangChain topluluk eklentileri için.
- `langchain-openai`: OpenAI modelleriyle LangChain entegrasyonu sağlar.
- `langchain_core`: LangChain’in temel bileşenleri için.
- `faiss-cpu`: Facebook AI tarafından geliştirilen **FAISS** kütüphanesi, büyük veri setlerinde **hızlı vektör aramaları** yapmak için kullanılır.


In [ ]:
!pip install -q openai
!pip install -q -U langchain
!pip install -q -U langchain langchain_community
!pip install -q -U langchain-openai
!pip install -q langchain_core
!pip install -q -U langchain_openai
!pip install -q faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.7/412.7 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4

## **2. Gerekli Kütüphanelerin Yüklenmesi**

**📌** Bu bölümde, **LLM, veri analizi, görselleştirme ve vektör tabanlı arama** için kullanılacak kütüphaneleri içe aktarıyoruz.

### **Önemli Kütüphaneler ve Kullanım Amaçları:**
- **LangChain Bileşenleri:**
  - `OpenAIEmbeddings`: Metinleri vektör haline getirmek için OpenAI’nin embedding modelini kullanır.
  - `ChatOpenAI`: OpenAI’nin sohbet modelleri ile etkileşim kurar.
  - `FAISS`: Büyük belge koleksiyonlarında **hızlı vektör tabanlı aramalar** yapar.
  - `RecursiveCharacterTextSplitter`: Metinleri küçük parçalara bölerek daha iyi işlenmesini sağlar.
  - `ChatPromptTemplate`: Chatbot'un kullanıcı girdisine uygun yanıt üretmesi için önceden tanımlı şablonlar oluşturur.
  - `create_retrieval_chain`: Bilgiyi alma ve yanıt üretme zincirini oluşturur.
  - `create_history_aware_retriever`: Sohbet geçmişine dayalı bilgi getirme işlemi yapar.

- **Diğer Python Modülleri:**
  - `os`: Çevresel değişkenlere erişim sağlar.
  - `re` ve `ast`: Metin işleme ve Python nesnelerine çeviri yapar.
  - `datetime`: Tarih ve saat bilgilerini işler.
  - `pandas` ve `numpy`: Veri analizi için kullanılır.
  - `matplotlib.pyplot` ve `seaborn`: **Veri görselleştirme** için kullanılır.
  - `Counter`: Verilerde kelime sayımı yapar.
  - `warnings`: Uyarıları gizlemek için kullanılır

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_history_aware_retriever
from langchain_core.messages import HumanMessage, AIMessage
import os
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import TextLoader
from langchain_core.messages import HumanMessage
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.output_parsers import StrOutputParser
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
import re
import ast
from datetime import datetime
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
plt.style.use("seaborn-v0_8-whitegrid")

import seaborn as sns
from collections import Counter

import warnings
warnings.filterwarnings("ignore")

## **3. Uçuş Verilerinin Yüklenmesi**

**📌** Bu bölümde, uçuş bilgilerini içeren bir Excel dosyasını **Pandas** kütüphanesi ile yüklüyoruz.


In [ ]:
flight_df = pd.read_excel("flight_with_headers.xlsx")
flight_df.head()

,flight_key,airline_code,departure_iata,arrival_iata,departure_date,departure_time,arrival_date,arrival_time,fare_desc,price_class,price_adt,price_chd,price_inf,total_taxes,total_surcharges,baggage
0,XR-5852,XR,FRA,HRG,2025-04-04,21:00:00,2025-04-05,01:55:00,EKO,O,343.98,343.98,NaN,0.0,0.0,0
1,XR-5852,XR,FRA,HRG,2025-04-04,21:00:00,2025-04-05,01:55:00,FLEX,O,373.98,373.98,NaN,0.0,0.0,23
2,XR-5852,XR,FRA,HRG,2025-04-04,21:00:00,2025-04-05,01:55:00,PREMIUM,O,403.98,403.98,NaN,0.0,0.0,40
3,XC-9282,XC,FRA,AYT,2025-07-11,14:50:00,2025-07-11,19:25:00,EKO,O,245.08,245.08,NaN,0.0,0.0,0
4,XC-9282,XC,FRA,AYT,2025-07-11,14:50:00,2025-07-11,19:25:00,FLEX,O,275.08,275.08,NaN,0.0,0.0,23


## **4. LLM**

### **Model 1: Chat bot modelleme**

#### **OpenAI API Anahtarı Tanımlama ve LLM Modeli Oluşturma (Retrieval Augmented Generation -RAG)**

**📌** Bu bölümde, **OpenAI'nin GPT modeli** kullanılarak bir dil modeli (LLM) oluşturuluyor ve bilgiye dayalı arama (retrieval) işlemleri için vektör tabanlı bir sistem kuruluyor.

In [ ]:
os.environ["OPENAI_API_KEY"] = "sk-None-****"


# Burada sohbet amacli kullanacagimiz ilk modelimizi taminliyoruz
llm = ChatOpenAI(model_name="gpt-4o-mini-2024-07-18", temperature=1)

# rag_file.txt dosyasından belgeler yüklenir.
file_path = "rag_file.txt"
loader = TextLoader(file_path)

# RAG dosyasi load ediliyor.
docs = loader.load()

# OpenAI’nin embedding modeli, metni sayısal bir vektör haline getirir.
embeddings = OpenAIEmbeddings()


# Belgeler küçük parçalara bölünerek, modelin daha iyi yanıt verebilmesi sağlanır.
text_splitter = RecursiveCharacterTextSplitter()

documents = text_splitter.split_documents(docs)

# FAISS, hızlı vektör arama yapabilen bir sistemdir. Belgeler vektör formatına dönüştürülerek arama yapılabilir hale getirilir.
vector = FAISS.from_documents(documents,embeddings)
retriever = vector.as_retriever()

### **Chatbot’un Dil Seçimi ve Yanıt Şablonu**

**📌** Bu bölümde, chatbot’un kullanıcı diline göre yanıt vermesini sağlayan **prompt şablonu** oluşturuluyor ve sohbet geçmişini baz alarak cevap üreten bir bilgi getirme zinciri kuruluyor.


* {context}: Chatbot’un nasıl yanıt vereceğini belirleyen sistem mesajı. Bu kisimda veriler rag_file.txt icerisinde belirtilmistir.
* {chat_history}: Önceki konuşmalar saklanarak daha tutarlı yanıtlar verilmesini sağlar.
* {input}: Kullanıcıdan gelen yeni mesaj.
Respond in
* {language}: Yanıtın kullanıcının seçtiği dilde olmasını sağlar.

In [ ]:
language = "Turkish" #bu kisimda kullanici bayraklari kullanarak dilini sececek


# Chat Prompt Tanımlama
prompt3 = ChatPromptTemplate.from_messages(
    [
        ("system","{context}"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user","{input}"),
        ("user", "Respond in {language}.")
    ]
)

# Chatbot, geçmiş konuşmaları dikkate alarak cevap üretir
retriever_chain = create_history_aware_retriever(
    llm,
    retriever,
    prompt3,
)


# Chatbot, geçmiş konuşmaları dikkate alarak cevap üretir
document_chain = create_stuff_documents_chain(llm,prompt3)
retriever_chain = create_retrieval_chain(retriever_chain, document_chain)

chat_history = [
    HumanMessage(content="Hello."),
    AIMessage(content="Hello.")
    ]

In [ ]:
# Modelin genel görevini tanimliyoruz. Ayrintilar rag_file.txt dosyasi icerisinde
context = "You are Airtuerk's AI bot. Response kindly to the costumers. "

### **Kullanıcı Girişi İşleme ve Yanıt Döndürme Fonksiyonu**

Bu bölümde, chatbot’un kullanıcı girişlerini işleyip uygun yanıtlar üretmesini sağlayan `user_response` fonksiyonu oluşturuluyor.

In [ ]:

chat_history = []

def user_response(user_input):
    try:
        response = retriever_chain.invoke(
            {
                "language" :language,
                "context":context,
                "input": user_input,
                "chat_history": chat_history
            }
        )
        chat_history.extend([HumanMessage(content=user_input), response["answer"]])
        return response["answer"]
    except Exception as e:
        return f"An error occurred: {str(e)}"

### **Model 2: Gerekli Bilgilerin Toplanmasi- Name Entity Recognition (NER) modeli**

**📌** Bu bölümde, chatbot'un uçuş bilgilerini anlamlandırmasını sağlamak için **IATA havalimanı kodları** ve **kullanıcı yanıtlarını saklayan bir sözlük (dictionary)** tanımlanıyor.

Bu görevi yapabilmek icin konusma icerisinden istedigimiz bilgileri toplayarak, bu bilgileri daha sonra kullanmak üzere bir dictionary olusturan bir model tasarliyoruz.

Modelimizin toplamasini istedigimiz bilgiler sunlardir:

    user_responses = {
        "departure": "",
        "destination": "",
        "start_date": "",
        "end_date": "",
        "return_start_date": "",
        "return_end_date": "",
        "one_way": True,
    }

Bu kisim istedigimiz senaryo dogrultusunda configure edilebilecek sekilde tasarlanmistir. Farkli bilgilere ihtiyac duyuldugunda bu kisimda yapilacak degisiklikler modelin davranisini etkileyecektir. Bu model bir prompt engineering methodu olan '1-shot learning' yöntemi kullanilarak, yani istedigimiz ciktidan 1 örnek tanimlanarak olusturulmustur.

In [ ]:

iata_airports = {
    'IST' : 'Istanbul Airport',
    'LHR' : 'London Heathrow Airport',
    'HRG': 'Hurghada International Airport',
    'AYT': 'Antalya Airport',
    'BJV': 'Milas-Bodrum Airport',
    'SSH': 'Sharm El Sheikh International Airport',
    'RZV': 'Rize-Artvin Airport',
    'DNZ': 'Denizli Çardak Airport',
    'DLM': 'Dalaman Airport',
    'ECN': 'Ercan International Airport',
    'EBL': 'Erbil International Airport',
    'GZT': 'Gaziantep Oğuzeli Airport',
    'ASR': 'Kayseri Erkilet Airport',
    'VKO': 'Vnukovo International Airport',
    'OGU': 'Ordu-Giresun Airport',
    'MQM': 'Mardin Airport',
    'ADB': 'Izmir Adnan Menderes Airport',
    'TIA': 'Tirana International Airport',
    'AMM': 'Queen Alia International Airport',
    'SCO': 'Aktau Airport',
    'SAW': 'Sabiha Gökçen International Airport',
    'KHI': 'Jinnah International Airport',
    'COV': 'Coventry Airport',
    'KWI': 'Kuwait International Airport',
    'RUH': 'King Khalid International Airport',
    'KUT': 'David the Builder Kutaisi International Airport',
    'NQZ': 'Nursultan Nazarbayev International Airport',
    'DXB': 'Dubai International Airport',
    'SHJ': 'Sharjah International Airport',
    'GYD': 'Heydar Aliyev International Airport',
    'HBE': 'Borg El Arab Airport',
    'TBZ': 'Tabriz International Airport',
    'ERZ': 'Erzurum Airport',
    'DOH': 'Hamad International Airport',
    'BEY': 'Beirut-Rafic Hariri International Airport',
    'TZX': 'Trabzon Airport',
    'ESB': 'Ankara Esenboğa Airport',
    'TBS': 'Tbilisi International Airport',
    'IKA': 'Imam Khomeini International Airport',
    'SPX': 'Sphinx International Airport',
    'PRN': 'Pristina International Airport',
    'CMN': 'Casablanca Mohammed V International Airport',
    'KYA': 'Konya Airport',
    'ATH': 'Athens International Airport',
    'ALA': 'Almaty International Airport',
    'SZF': 'Samsun Çarşamba Airport',
    'EDO': 'Balıkesir Koca Seyit Airport',
    'JED': 'King Abdulaziz International Airport',
    'ADF': 'Adıyaman Airport',
    'EVN': 'Zvartnots International Airport',
    'MED': 'Prince Mohammad bin Abdulaziz Airport',
    'BGW': 'Baghdad International Airport',
    'RMO': 'Rome Fiumicino Airport',
    'SKP': 'Skopje International Airport',
    'DIY': 'Diyarbakır Airport',
    'MCT': 'Muscat International Airport',
    'KCM': 'Kahramanmaraş Airport',
    'MLX': 'Malatya Erhaç Airport',
    'CIT': 'Shymkent International Airport',
    'DMM': 'King Fahd International Airport',
    'BAH': 'Bahrain International Airport',
    'AUH': 'Abu Dhabi International Airport',
    'BGG': 'Bingöl Airport',
    'SJJ': 'Sarajevo International Airport',
    'MSR': 'Muş Airport',
    'KSY': 'Kars Harakani Airport',
    'GNY': 'Şanlıurfa GAP Airport',
    'TZL': 'Tuzla International Airport',
    'IFN': 'Isfahan International Airport',
    'NOP': 'Sinop Airport',
    'EZS': 'Elazığ Airport',
    'BAL': 'Batman Airport',
    'FRU': 'Bishkek Manas International Airport',
    'LED': 'Pulkovo Airport (St. Petersburg)',
    'IGD': 'Iğdır Airport',
    'SOF': 'Sofia Airport',
    'VAN': 'Van Ferit Melen Airport',
    'ERC': 'Erzincan Airport',
    'MHD': 'Mashhad International Airport',
    'BSR': 'Basra International Airport',
    'OSS': 'Osh Airport',
    'MZH': 'Amasya Merzifon Airport',
    'GZP': 'Gazipaşa-Alanya Airport',
    'VAS': 'Sivas Nuri Demirağ Airport',
    'OTP': 'Bucharest Henri Coandă International Airport',
    'SYZ': 'Shiraz International Airport',
    'BER': 'Berlin Brandenburg Airport',
    'MUC': 'Munich Airport',
    'FRA': 'Frankfurt Airport',
}



# Kullanıcı yanıtlarını saklayacak dictionary
user_responses = {
    "departure": "",
    "destination": "",
    "start_date": "",
    "end_date": "",
    "return_start_date": "",
    "return_end_date": "",
    "one_way": True,
}

# Model
user_data_collector = ChatOpenAI(model_name="gpt-4o-mini-2024-07-18", temperature=0.1)

# Prompt tanımı
prompt_data_collector = """
You are an AI assistant designed to extract key information from user responses and update the corresponding fields in a dictionary.

### **Instructions:**
- Identify if the user mentions **departure airport, destination airport, travel dates, or ticket type (one-way/round-trip).**
- If the user provides a **city**, find the corresponding **major airport's IATA code**.
- If the user provides an **IATA airport code**, use it directly.
- Use the provided **airport mapping table** to ensure correct IATA codes.
- If a value is already set and the new information contradicts it, replace the old value with the new one.
- If user says return date, update relevant filed and also set one_way to False (always capitalize the first letter).
---
- Extract travel dates and store them in **DD/MM/YYYY format**.
- If the user mentions **one-way**, set `"one_way": true` and clear return dates.
- If the user provides a **date without a year**, assume the current year `{current_year}`, now 2025.
- Use the provided **airport mapping table** to ensure correct IATA codes.
- Update only the relevant field. Leave the other information unchanged.

### **Fields to Update:**
- `"departure"`: The user's departure **airport IATA code**
- `"destination"`: The user's destination **airport IATA code**
- `"start_date"`: The user's departure date **(format: DD/MM/YYYY)**
- `"end_date"`: The user's last available departure date (for flexible searches)
- `"return_start_date"`: The user's return date **(format: DD/MM/YYYY, if applicable)**
- `"return_end_date"`: The user's last available return date (for flexible searches)
- `"one_way"`: `true` if the user selects a one-way ticket

---

"""


prompt_list_generation = ChatPromptTemplate.from_template(
    """{prompt_data_collector}

    I will give you a user response in a dictionary format. Here is the current dictionary: {user_responses}

    Analyze the conversation below and update the dictionary accordingly:

    **User Response:** {conversation}

      ### **Airport Mapping Table (Airport Names → IATA Codes ):**
      {iata_airports}


    **Output Format:**
    Return definitely and only a dictionary with the following structure, and not JSON format. I need python dict format:

    {{
        "departure": "",
        "destination": "",
        "start_date": "",
        "end_date": "",
        "return_start_date": "",
        "return_end_date": "",
        "one_way": True
    }}



    ### **Example 1:**
    **User Input:** "I am flying from Berlin."
    Output:
    {{
        "departure": "BER",
        "destination": "",
        "start_date": "",
        "end_date": "",
        "return_start_date": "",
        "return_end_date": "",
        "one_way": True
    }}
    **User Input:** "I am going to Istanbul."
    Output:
    {{
        "departure": "BER",
        "destination": "IST",
        "start_date": "",
        "end_date": "",
        "return_start_date": "",
        "return_end_date": "",
        "one_way": True
    }}
    **User Input:** "I am planning between 15 May and 21 May."
    Output:
    {{
        "departure": "BER",
        "destination": "IST",
        "start_date": "15/05/2025",
        "end_date": "21/05/2025",
        "return_start_date": "",
        "return_end_date": "",
        "one_way": True
    }}
    **User Input:** "It is one way."
    Output:
    {{
        "departure": "BER",
        "destination": "IST",
        "start_date": "15/05/2025",
        "end_date": "21/05/2025",
        "return_start_date": NaN,
        "return_end_date": NaN,
        "one_way": True
    }}
        """
)

# Çıktı işleyici
output_parser = StrOutputParser()

# Zincirleme işlem
chain_list_generation = prompt_list_generation | user_data_collector | output_parser

# Çalıştırma fonksiyonu
def generate_dict(conversation, user_responses):
    try:
        # Modeli çalıştır ve güncellenmiş dictionary'yi al
        updated_user_responses = chain_list_generation.invoke({
            "prompt_data_collector": prompt_data_collector,
            "conversation": conversation,
            "iata_airports": iata_airports,
            "user_responses": str(user_responses)  # JSON string olarak geçiriyoruz
        })

        return updated_user_responses
    except Exception as e:
        return f"An error occurred: {str(e)}"



In [ ]:
# Bu fonksiyon NER modelinden(user_data_collector) string halde gelen bilgiyi dictionary haline getirmektedir.

def convert2dict(raw_string):
  # Cleaning code blocks and convert into dictionary
  cleaned_string = re.sub(r"^```python", "", raw_string.strip())
  cleaned_string = re.sub(r"```$", "", cleaned_string.strip(), flags=re.MULTILINE).strip()
  try:
    user_dict = ast.literal_eval(cleaned_string)
  except Exception as e:
    print("Converting Error:", str(e))

  return user_dict

## **Model 3: Uçuş Arama Sorgusunun Tespiti - Classifier Model Tanimlamasi**

**📌** Bu bölümde, **kullanıcının uçuş arama niyetini** anlamak için bir sınıflandırma modeli oluşturuluyor.  <br />

gpt-4o-mini modeli kullanılarak düşük sıcaklıkta (temperature=0.1) daha tutarlı ve kesin yanıtlar üretmesi sağlanıyor.  <br />

Model özetle: <br />

* Kullanıcının uçuş arama isteğinde bulunup bulunmadığını belirlemek için bir prompt (metin şablonu) oluşturulur.
* "True" döndürmesi için mesajın açıkça uçuş arama niyetinde olması gerekir.
* "False" döndürmesi için mesajın uçuş aramayla ilgili olmaması gerekir.

In [ ]:
# model tanimlama

model_search_flight = ChatOpenAI(model_name="gpt-4o-mini-2024-07-18", temperature=0.1)

prompt_search_flight = ChatPromptTemplate.from_template(
        """You are a classification model that reads a single message in a ticket sales context.

Output exactly "True" if this specific message clearly indicates that
the speaker is about to or already searching for flights (ex: "I am looking for flights", "I'll find the options right now",
"Uçuşlara şimdi bakıyorum", "Hemen kontrol ediyorum" etc.).

Otherwise, output exactly "False". No explanations.

Examples:
- "I will search for flights for you." => True
- "We need your passport details" => False
- "Tamam, hemen bakıyorum." => True
- "Ne zaman gitmek istersiniz?" => False

The message is: {message}
"""
)

output_parser = StrOutputParser()
search_flight_chain = prompt_search_flight | model_search_flight | output_parser

# Uçuş Arama Durumunu Kontrol Etme
def search_flight(user_input):
    try:
        # Perform comparison
        response_for_searching = search_flight_chain.invoke({"message": user_input})

        return ast.literal_eval(response_for_searching)
    except Exception as e:
        return f"An error occurred: {str(e)}"

## **5. Tarih İşleme ve Uçuşları Filtreleme**
**📌**
Bu bölümde, kullanıcının belirttiği tarih ve lokasyonlara uygun uçuşları filtreleyen fonksiyonlar oluşturuluyor. <br />

Bu kisimda yapilanlar su sekildedir:

* Tarih Verilerini İşle
* Kullanıcının Girdiği Tarihleri Al
* Gidiş Uçuşlarını Filtreleme
* Tek Yön veya Gidiş-Dönüş Seçimi
* Gidiş-Dönüş Uçuşlarını Filtreleme

In [ ]:
# tarih verilerini isle
def parse_date(date_str):

    # "15/05/2025" → datetime(2025, 5, 15)
    return datetime.strptime(date_str, "%d/%m/%Y").date()

# ucuslari filtrele
def filter_flights(df, user_dict):
    """
    Filter dataframe regarding user infos.
    """

    df["departure_date"] = pd.to_datetime(df["departure_date"]).dt.date
    df["arrival_date"] = pd.to_datetime(df["arrival_date"]).dt.date


    if user_dict["start_date"]:
        start_date = parse_date(user_dict["start_date"])
    else:
        start_date = None

    if user_dict["end_date"]:
        end_date = parse_date(user_dict["end_date"])
    else:
        end_date = start_date

    # Filter deparature
    df_go = df[
        (df["departure_iata"] == user_dict["departure"]) &
        (df["arrival_iata"] == user_dict["destination"])
    ]
    # Filter based date
    if start_date is not None:
        df_go = df_go[(df_go["departure_date"] >= start_date) & (df_go["departure_date"] <= end_date)]

    if user_dict["one_way"]:
        # One-way or another
        return df_go, None
    else:
        # Not one way
        if user_dict["return_start_date"]:
            ret_start = parse_date(user_dict["return_start_date"])
        else:
            ret_start = None
        if user_dict["return_end_date"]:
            ret_end = parse_date(user_dict["return_end_date"])
        else:
            ret_end = ret_start

        # filter return flights
        df_return = df[
            (df["departure_iata"] == user_dict["destination"]) &
            (df["arrival_iata"] == user_dict["departure"])
        ]

        if ret_start is not None:
            df_return = df_return[
                (df_return["departure_date"] >= ret_start) &
                (df_return["departure_date"] <= ret_end)
            ]

        return df_go, df_return


In [ ]:
# Test icin örnek veri olusturup yukaridaki fonksiyonlari deniyoruz.

df = flight_df
user_info = {
    "departure": "FRA",
    "destination": "HRG",
    "start_date": "01/05/2025",
    "end_date": "",
    "return_start_date": "",
    "return_end_date": "",
    "one_way": True
}

In [ ]:
# filtre fonksiyonu test
df_gidis, df_donus = filter_flights(df,user_info)

# Sonuçları ekrana yaz
print("=== Gidiş Uçuşları ===")
if df_gidis is not None and not df_gidis.empty:
    print(df_gidis)
else:
    print("Uygun gidiş uçuşu bulunamadı.")

if user_info["one_way"] == False:
    print("\n=== Dönüş Uçuşları ===")
    if df_donus is not None and not df_donus.empty:
        print(df_donus)
    else:
        print("Uygun dönüş uçuşu bulunamadı.")

=== Gidiş Uçuşları ===
          flight_key airline_code departure_iata arrival_iata departure_date  \
17903  PC-994_PC-620           PC            FRA          HRG     2025-05-01   
31398  PC-994_PC-622           PC            FRA          HRG     2025-05-01   
37006  PC-996_PC-622           PC            FRA          HRG     2025-05-01   

      departure_time arrival_date arrival_time fare_desc price_class  \
17903       14:45:00   2025-05-02     01:45:00     SAVER           W   
31398       14:45:00   2025-05-02     02:55:00     SAVER           W   
37006       17:45:00   2025-05-02     02:55:00     SAVER           W   

       price_adt  price_chd  price_inf  total_taxes  total_surcharges  baggage  
17903      231.9      231.9       51.0          NaN               NaN       20  
31398      231.9      231.9       51.0          NaN               NaN       20  
37006      231.9      231.9       51.0          NaN               NaN       20  


# **10. Gerçek Zamanlı Chatbot - Modellerin Entegrasyonu ve Test Edilmesi**

Bu bölümde, **gerçek zamanlı bir chatbot** oluşturularak, kullanıcının girişine yanıt verebilen ve uygun uçuşları listeleyebilen **uçtan uca çalışan bir sistem** kuruluyor.

---

## **📌 Kullanılan 3 Model**
Chatbot’un çalışması için **üç temel model** entegre edilmiştir:

1. **Dil Modeli (LLM - GPT-4o Mini)**
   - Kullanıcının mesajlarını işleyip uygun yanıtlar üretir.
   - **Fonksiyon:** `user_response(user_input)`

2. **Uçuş Arama Modeli**
   - Kullanıcının **uçuş arama isteğinde bulunup bulunmadığını** tespit eder.
   - **Fonksiyon:** `search_flight(user_input)`

3. **Uçuş Filtreleme Modeli**
   - Kullanıcının belirttiği **kalkış, varış ve tarih bilgilerine** göre uçuş listesini filtreler.
   - **Fonksiyon:** `filter_flights(df, user_dict)`

* Sohbet geçmişi (chat_history) başlatılır.
* Kullanıcının yanıtlarını tutacak bir JSON sözlüğü (user_responses) oluşturulur.
* Kullanıcıdan sürekli giriş alacak bir döngü başlatılır.
* Uçuş Arama Modeli Çalıştırılır
* Uygun Uçuşlar Listelenir
* Chatbot Yanıtı Üretilir.
* Kullanıcı Yanıtları Güncellenir.

      **Örnek calisma senaryosu:**

      Welcome to Airtuerk AI Bot! Type 'exit' to quit.

      **You**: I am looking for flights from Berlin to Istanbul.
      **Bot**: Sure! When would you like to travel?

      **You**: I want to go on May 10th.
      **Bot**: One-way or round trip?

      **You**: One-way.
      === Gidiş Uçuşları ===
      departure   _iata    arrival_iata   departure_date   price
      0             BER          IST        10/05/2025      150€
    
      user_responses:
      {
          "departure": "BER",
          "destination": "IST",
          "start_date": "10/05/2025",
          "end_date": "",
          "return_start_date": "",
          "return_end_date": "",
          "one_way": True
      }

In [ ]:
# Kullanıcıdan canlı giriş alan döngü

chat_history = ["Chat Bot Speaking..."]

user_responses = """{
    "departure": "",
    "destination": "",
    "start_date": "",
    "end_date": "",
    "return_start_date": "",
    "return_end_date": "",
    "one_way": True,
}"""

print("Welcome to Airtuerk AI Bot! Type 'exit' to quit.\n")

while True:
    user_input = input("You: ")  # Kullanıcıdan input al
    if (search_flight(chat_history[-1]) == True):  # Çıkış komutu Classifier modeli tarafindan kontrol ediliyor
        df_gidis, df_donus = filter_flights(df,convert2dict(user_responses))

        # Sonuçları ekrana yaz
        print("=== Gidiş Uçuşları ===")
        if df_gidis is not None and not df_gidis.empty:
            print(pd.DataFrame(df_gidis))
        else:
            print("Uygun gidiş uçuşu bulunamadı.")

        if user_info["one_way"] == False:
            print("\n=== Dönüş Uçuşları ===")
            if df_donus is not None and not df_donus.empty:
                print(df_donus)
            else:
                print("Uygun dönüş uçuşu bulunamadı.")

    bot_response = user_response(user_input)  # Bot yanıtını al
    print(f"Bot: {bot_response}\n")  # Yanıtı ekrana yazdır

    user_responses = generate_dict(chat_history, user_responses)
    print(f"User Responses: {user_responses}\n")


Welcome to Airtuerk AI Bot! Type 'exit' to quit.

Bot: Merhaba! Size nasıl yardımcı olabilirim?

User Responses: {
    "departure": "",
    "destination": "",
    "start_date": "",
    "end_date": "",
    "return_start_date": "",
    "return_end_date": "",
    "one_way": True
}

Bot: Harika! Hangi noktadan hareket edeceksiniz?

User Responses: {
    "departure": "",
    "destination": "IST",
    "start_date": "",
    "end_date": "",
    "return_start_date": "",
    "return_end_date": "",
    "one_way": True
}

Bot: Tamam, Antalya'ya gitmek istiyorsunuz. Hangi tarihte hareket etmeyi planlıyorsunuz? (Bir tarih aralığı da verebilirsiniz.)

User Responses: ```python
{
    "departure": "FRA",
    "destination": "AYT",
    "start_date": "",
    "end_date": "",
    "return_start_date": "",
    "return_end_date": "",
    "one_way": True
}
```

Bot: Tamam! 11 Temmuz'da Antalya'ya gitmeyi planlıyorsunuz. Peki, dönüş tarihiniz ne zaman olacak? (Bir tarih aralığı da verebilirsiniz.)

User Response

KeyboardInterrupt: Interrupted by user